# **Importing libraries**

In [ ]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
from sklearn.utils import shuffle
import backtrader as bt
import math
import ta
from statistics import mean
import time
import datetime
import matplotlib.pyplot as plt
import matplotlib
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , BatchNormalization , GRU ,LeakyReLU
from tensorflow.keras.layers import LSTM , Conv1D
from tensorflow.keras.layers import Dropout ,Flatten 
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import RootMeanSquaredError
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint , TensorBoard
# from imblearn.under_sampling import ClusterCentroids

# **Loading Dataset**

In [ ]:
base_dir = os.getcwd()
minute_data = os.path.join(base_dir,'eurusd_minute.csv')

In [ ]:
df = pd.read_csv(minute_data,usecols = ['Date', 'Time' ,'BO','BH','BL','BC'], parse_dates = [[0, 1]], date_parser=lambda col: pd.to_datetime(col, utc=False),nrows = 100000)

# **Calculate Signal**

In [ ]:
pip = 10
timeframe = 30
lookback_period  = 100
BATCH_SIZE = 32
lstm_units = 7
lr = 0.000001
epoches = 300
dx = 	pip/10000
df['TimeframeDiff'] = df['BC'] - df['BC'].shift(timeframe)
df['TimeframeDiff'] = df['TimeframeDiff'].shift(-timeframe,fill_value = 0)
df['Signal'] = np.select([df['TimeframeDiff'] > dx,df['TimeframeDiff']<-dx],[1,-1],default = 0)

# **Import Features**

In [ ]:
from ta.trend import sma_indicator as sma
sma_close = sma(df['BC'] , 10,fillna= True)

In [ ]:
from ta.trend import macd
macd_close = macd(df['BC'],window_fast = 12, window_slow = 26,fillna = True)

In [ ]:
from ta.trend import cci
cci_close = cci(df['BH'],df['BL'], df['BC'] , window = 20 , fillna = True)

In [ ]:
from ta.momentum import roc
roc_close = roc(df['BC'],window = 2,fillna= True)

In [ ]:
from ta.momentum import rsi
rsi_close  = rsi(df['BC'] ,window = 14, fillna = True) 

In [ ]:
from ta.volatility import bollinger_mavg as bb
bb_close  = bb(df['BC'] , window = 20 , fillna = True)

In [ ]:
feature_list  = [df['TimeframeDiff'],sma_close , macd_close , roc_close , rsi_close , bb_close , cci_close ]

# **Combining all features together to input array**

In [ ]:
Combination = pd.concat(feature_list,axis = 1)

Combination.fillna(0, inplace=True)

In [ ]:
Combination_array = pd.DataFrame.to_numpy(Combination)

n_features = Combination_array.shape[1]

In [ ]:
data = Combination_array

In [ ]:
plot_data = data[:, 0]

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.ylabel("TimeframeDiff")
plt.xlabel("Time")
plt.plot(plot_data, color = 'blue', label = 'Training Set')
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(data[:,0])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

# **Function for generating data with sliding window**


In [ ]:
def data_generator_diff(data, signal,start, finish, lookback_period , timeframe ,dx):
    data = data[start:finish]
    
    X_data = []
    y_data = []
    
    for i in range(0, len(data)-lookback_period-timeframe+1):
        X_data.append(data[i:i+lookback_period-1, :n_features]) 
        if signal[i+lookback_period] == 1:
            y_data.append(2)
        elif signal[i+lookback_period] == -1:
            y_data.append(1)
        else:
            y_data.append(0) 
    
    X_data , y_data = np.array(X_data), np.array(y_data)

        
    X_data = np.reshape(X_data, (X_data.shape[0], X_data.shape[1], n_features)) 
    
    sc = MinMaxScaler(feature_range = (0, 1))
    
    for i in range(X_data.shape[0]):
        temp = X_data[i,:,0].reshape(-1,1)
        temp = sc.fit_transform(temp)
        X_data[i,:,0] = temp[:,0]
    
    return X_data , y_data

In [ ]:
signal = df['Signal'].values

# **Generate train set and test set**

In [ ]:
train_start = 0
train_end = math.floor(0.8*Combination_array.shape[0])
val_start = train_end
val_end = math.floor(0.9*Combination_array.shape[0])
test_start = val_end
test_end = Combination_array.shape[0]
X_train , y_train = data_generator_diff(data,signal, train_start, train_end , lookback_period ,timeframe , dx)
X_val , y_val  = data_generator_diff(data,signal, val_start, val_end , lookback_period ,timeframe , dx)
X_test , y_test = data_generator_diff(data,signal, test_start, test_end, lookback_period ,timeframe,dx)

# **Shuffle train data**

In [ ]:
X_train, y_train = shuffle(X_train, y_train)

# **Plotting**

# **Plotting train and test set**

In [ ]:
train_set = df.iloc[train_start:train_end , [3]]

fig = plt.figure(figsize=(20,10))
plt.ylabel("SPY")
plt.xlabel("Time")
plt.plot(train_set, color = 'blue', label = 'Training Set')
plt.show()

In [ ]:
test_set = df.iloc[test_start:test_end , [3]]

fig = plt.figure(figsize=(20,10))
plt.ylabel("SPY")
plt.xlabel("Time")
plt.plot(test_set, color = 'red', label = 'Test Set')
plt.show()

# **Training the network**

# **Model Architecture**

In [ ]:
model = Sequential()

model.add(LSTM(units= lstm_units,input_shape = (X_train.shape[1],n_features)))
model.add(Dropout(0.1))

model.add(Dense(units = 3,activation = 'softmax'))

opt = tf.keras.optimizers.Adam(learning_rate= lr)

model.compile(optimizer = opt, loss='sparse_categorical_crossentropy', metrics=['acc'])

print(model.summary())

es = EarlyStopping(monitor = 'val_acc', mode= 'auto',patience=10)

my_callbacks = [es]

history = model.fit(X_train, y_train 
          ,epochs = epoches
          ,batch_size = BATCH_SIZE
          ,validation_data=(X_val, y_val)
          ,callbacks = my_callbacks
          )

model.save('/content/drive/MyDrive/agr/Models/my_model.h5')

# **Plotting losses and accuracy**

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# **Accuracy-Loss on Test set**

In [ ]:
results = model.evaluate(X_test, y_test, batch_size= BATCH_SIZE)
print("test loss, test acc:", results)